In [1]:
%matplotlib inline
import os
if os.path.exists("myScripts/dataingestion.py"):
    os.remove("myScripts/dataingestion.py")

In [2]:
%%writefile myScripts/dataingestion.py -a 
import pandas as pd
import numpy as np

Writing myScripts/dataingestion.py


In [3]:
%%writefile myScripts/dataingestion.py -a

## Merge two dataframes in range
def merge_on_range(A, B):
    '''
    Args: 
        Dataframe A consists of the ip_addresses associated with each transaction
        Dataframe B consists of lower and upper bounds of ip_addresses corresponding to each country
        
    Output:
        Dataframe that uniquely maps each ip_address to a 
        specific country and returns a null if a match is not found
    '''
    a = A.ip_address.values
    bh = B.upper_bound_ip_address.astype(float).values
    bl = B.lower_bound_ip_address.astype(float).values

    i, j = np.where((a[:, None] >= bl) & (a[:, None] <= bh))

    return pd.DataFrame(
        np.column_stack([A.values[i], B.values[j]]),
        columns=A.columns.append(B.columns)
    ).drop(['lower_bound_ip_address', 'upper_bound_ip_address'], axis = 1)

def join_datasets(fraud_datapath, IPtoCountry_datapath, output_path):
    '''
    Args: 
        fraud_datapath: path to the csv file containing dataset with all transactions
        IPtoCountry_datapath: path to the xlsx file with lower and upper bounds of ip_addresses for each country
        output_path: path for the output of mapping function
    Output:
        Dataframe that uniquely maps each ip_address to a specific country
        Dataframe is also saved as a csv file in the output_path
    '''
    fraud_df = pd.read_csv(fraud_datapath).drop('Unnamed: 0', axis = 1)
    ip_mapping_df = pd.read_excel(IPtoCountry_datapath)
    df_joined = merge_on_range(fraud_df, ip_mapping_df)

    ## Some IPs do not fall within any range
    country_na = fraud_df[~fraud_df.user_id.isin(df_joined.user_id)]

    ## Bring them in to the analysis dataset with missing country information
    analysis_df = pd.concat([df_joined, country_na], sort = False).fillna('NA')
    analysis_df.to_csv(output_path)
    
    return analysis_df

Appending to myScripts/dataingestion.py


In [4]:
## Run once
from myScripts.dataingestion import join_datasets
analysis_df = join_datasets(fraud_datapath = 'Candidate_tech_evaluation_candidate_copy_data science_fraud.csv',
                            IPtoCountry_datapath = 'Candidate_tech_evaluation_candidate_copy_datascience_IpAddress_to_Country.xlsx', 
                            output_path = 'Analysis_dataset.csv')

In [5]:
analysis_df.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country
0,285108,7/15/2015 4:36,9/10/2015 14:17,31,HZAKVUFTDOSFD,Direct,Chrome,M,49,2.818400e+09,0,United States
1,131009,1/24/2015 12:29,4/13/2015 4:53,31,XGQAJSOUJIZCC,SEO,IE,F,21,3.251268e+09,0,United Kingdom
2,328855,3/11/2015 0:54,4/5/2015 12:23,16,VCCTAYDCWKZIY,Direct,IE,M,26,2.727760e+09,0,United States
3,229053,1/7/2015 13:19,1/9/2015 10:12,29,MFFIHYNXCJLEY,SEO,Chrome,M,34,2.083420e+09,0,Korea Republic of
4,108439,2/8/2015 21:11,4/9/2015 14:26,26,WMSXWGVPNIFBM,Ads,FireFox,M,33,3.207913e+09,0,Brazil
